In [1]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import json

# Load data and GeoJSON
data = pd.read_csv(r'../data/LinkedIn_Tech_Jobs.csv')
with open('../data/india_state_geo.json', 'r') as file:
    india_geojson = json.load(file)

# Trim whitespace and standardize names
data['Location'] = data['Location'].str.strip()
# Replace 'Orissa' with 'Odisha' in the data if necessary
data['Location'] = data['Location'].replace({'Orissa': 'Odisha'})

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': x, 'value': x} for x in data['Company_Name'].unique()],
        value=data['Company_Name'].unique()[0]
    ),
    dcc.Graph(id='map-visualization')
])

@app.callback(
    Output('map-visualization', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_map(selected_company):
    filtered_data = data[data['Company_Name'] == selected_company]
    
    fig = px.choropleth_mapbox(filtered_data, geojson=india_geojson, color="Total_applicants",
                               locations="Location", featureidkey="properties.NAME_1",
                               center={"lat": 22.5937, "lon": 78.9629},
                               mapbox_style="carto-positron", zoom=4,
                               hover_data=["Designation", "Total_applicants"])
    fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
